In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from ast import literal_eval
from pathlib import Path

In [4]:
from sglxarray import load_trigger
import ecephys_analyses as ea
from ecephys_analyses.load import load_sr_chans
import hypnogram as hg

In [5]:
def add_duration(df):
    df = df.copy()
    df["duration"] = df.apply(lambda row: row.end_time - row.start_time, axis=1)
    return df

def write_experiment(subject, experiment):
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.csv")
    sr_chans_dt_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.tsv")
    
    all_sr_chans = [load_sr_chans(path) for path in sr_chans_paths]
    sigs = [load_trigger(path, chans=[0], start_time=0, end_time=1) for path in bin_paths]
        
    for sig, sr_chans, sr_chans_dt_path in zip(sigs, all_sr_chans, sr_chans_dt_paths):
        file_start_dt = pd.to_datetime(sig.datetime.values.min())
        sr_chans_dt = hg.FloatHypnogram(sr_chans)
        sr_chans_dt = add_duration(sr_chans_dt)
        sr_chans_dt.as_datetime(file_start_dt).write(sr_chans_dt_path)

In [10]:
write_experiment("Alessandro", "dexmedetomidine-eStim")

nChan: 385, nFileSamp: 7122380
nChan: 385, nFileSamp: 3753750
